### Dota Plus Post Message in Social Feed

Valve removed interface to send messages in your profile Social Feed but GameCoordinator Proto messages are still there

Change `MSG` to what you want to post and `STEAM_LGN`, `STEAM_PSW` pair in `.env` file.

In [ ]:
from steam.client import SteamClient
from dota2.client import Dota2Client
from dota2.enums import EDOTAGCMsg

import os
from dotenv import load_dotenv
load_dotenv(dotenv_path='env.env', verbose=True)
STEAM_LGN = os.getenv("STEAM_LGN") # type: ignore
STEAM_PSW = os.getenv("STEAM_PSW") # type: ignore

MSG = "You can see me drinking Cherry Cola... Sweet serial killer, I left a love note..."

class TestDota:
    def __init__(self, client, dota):
        self.client = client
        self.dota = dota
        self._StartDota = self.client.on('logged_on')(self.StartDota)
        self._LobbyLoop = self.dota.on('ready')(self.mypc)
        self.mypc_response = self.dota.on(EDOTAGCMsg.EMsgGCToClientSocialFeedPostMessageResponse)(self.__handle_mypc)

    def mypc(self):
        print(MSG)
        self.dota.send(EDOTAGCMsg.EMsgClientToGCSocialFeedPostMessageRequest, {'message': MSG})

    def __handle_mypc(self, message):
        print(message)
        self.dota.emit('my_message_sent', message)

    def StartDota(self):
        print('Starting Dota GC communication')
        self.dota.launch()


client = SteamClient()
dota = Dota2Client(client)

test_dota = TestDota(client, dota)
client.cli_login(username=STEAM_LGN, password=STEAM_PSW)
client.run_forever()

### Utils for Dota 2 things 

this is toned down non-async copy of code from my discord bot ; \
but we also need it for some sandbox snippets

In [5]:
import requests


def opendota_constants_heroes():
    endpoint = 'https://api.opendota.com/api/constants/heroes'
    response = requests.get(endpoint)
    dic = response.json()

    data = {
        'id_by_npcname':
            {'': 0},
        'id_by_name':
            {'bot_game': 0},
        'name_by_id':
            {0: 'bot_game'},
        'iconurl_by_id':
            {0: "https://static.wikia.nocookie.net/dota2_gamepedia/images/3/3d/Greater_Mango_icon.png"}
    }
    for hero in dic:
        data['id_by_npcname'][dic[hero]['name']] = dic[hero]['id']
        data['id_by_name'][dic[hero]['localized_name']] = dic[hero]['id']
        data['name_by_id'][dic[hero]['id']] = dic[hero]['localized_name']
        data['iconurl_by_id'][dic[hero]['id']] = f"https://cdn.cloudflare.steamstatic.com/{dic[hero]['img']}"
    return data

hero_keys_cache = opendota_constants_heroes()

async def id_by_npcname(value: str) -> int:
    """ Get hero id by npcname ;
    example: 'npc_dota_hero_antimage' -> 1 """
    return hero_keys_cache['id_by_npcname'][value]


async def id_by_name(value: str) -> int:
    """ Get hero id by localized to english name ;
    example: 'Anti-Mage' -> 1 """
    return hero_keys_cache['id_by_name'][value]


async def name_by_id(value: int) -> str:
    """ Get hero id by name ;
    example: 1 -> 'Anti-Mage' """
    return hero_keys_cache['name_by_id'][value]


async def iconurl_by_id(value: int) -> str:
    """ Get hero icon utl id by id ;
    example: 1 -> 'https://cdn.cloudflare.steamstatic.com//apps/dota2/images/dota_react/heroes/antimage.png?' """
    return hero_keys_cache['iconurl_by_id'][value]

# How many heroes are in Dota ?
len(hero_keys_cache['id_by_name']) - 1 # minus one for that zero value

123

### Dota Hero Grid Editor

Json file in steam folder:\
`C:\Program Files (x86)\Steam\userdata\YOUR_FRIEND_ID\570\remote\cfg\hero_grid_config.json`

Some useful info:
```python
max_x = 1200
max_y = 598
```
`data['configs'][0]` is my main grid\
`data['configs'][1]` is Dota Plus levels grid

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path='env.env', verbose=True)

YOUR_FRIEND_ID = os.getenv('DOTA_FRIENDID')
print(f'C:\\Program Files (x86)\\Steam\\userdata\\{YOUR_FRIEND_ID}\\570\\remote\\cfg\\hero_grid_config.json')

# do not forget to clear output

In [ ]:
import json
# READ FROM FILE
with open('hero_grid_config.json') as json_file:
    data = json.load(json_file)

In [196]:
# WRITE INTO FILE 
with open('hero_grid_config.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

### Dota Plus Levels Grid

In [195]:
dict_help = {
    'Grandmaster': [0, 0, 300, 100],
    'Master':      [300, 0, 300, 100],
    'Platinum':    [600, 0, 600, 100],
    'Gold':        [0, 115, 600, 100],
    'Silver':      [600, 115, 600, 100],
    'Bronze':      [0, 230, 1200, 275],
    'Level 0':     [0, 530, 1200, 68],
}

for cat in data['configs'][1]['categories']:
    array = dict_help[cat['category_name']]
    cat['x_position'] = array[0]
    cat['y_position'] = array[1]
    cat['width'] = array[2]
    cat['height'] = array[3]

In [189]:
data['configs'][0]['categories'][-1]

{'category_name': 'bans',
 'x_position': 925,
 'y_position': 529.0,
 'width': 275,
 'height': 69.0,
 'hero_ids': [137, 88]}

### My default picking screen Grid

In [ ]:
names = []
for i in data['configs'][0]['categories'][0]['hero_ids']:
    names.append(name_by_id(i))
    
new_names = sorted(names, key=str.casefold)

new_ids = []
for i in new_names:
    new_ids.append(id_by_name(i))
    
data['configs'][0]['categories'][0]['hero_ids'] = new_ids
print(new_ids)

In [ ]:
max_x = 1200
max_y = 598

height = 100
delta_y = (max_y - 5 * height)/4

pos_w = 7 * 58 # 7 for seven heroes in pos 1
others_x = pos_w + 1

counter = 0
for item in data['configs'][0]['categories'][1:]:
    item['x_position'] = 0
    item['y_position'] = 0 + (height + delta_y) * counter
    item['height'] = height
    item['width'] = pos_w
    #print(item['category_name'], len(item['hero_ids']), item['hero_ids'])
    counter += 1
    
last_item = data['configs'][0]['categories'][0]
last_item['x_position'] = others_x 
last_item['y_position'] = 0
last_item['height'] = max_y
last_item['width'] = max_x - others_x 